# Accessibility Improvements for the Disabled

In [86]:
#importing dependencies
import speech_recognition as sr
import time
from gtts import gTTS
from playsound import playsound
# from pydub import AudioSegment
import os

In [87]:
recognizer = sr.Recognizer()

In [88]:
def capture_voice_input():
    with sr.Microphone() as source:
        print("Listening...")
        recognizer.adjust_for_ambient_noise(source)

        try:
            audio = recognizer.listen(source, timeout=5, phrase_time_limit=5)
        except sr.WaitTimeoutError:
            print("Listening timed out while waiting for you to speak")
            return None
    return audio

In [89]:
def text_to_speech(text):
    # Initialize gTTS with the text to convert
    speech = gTTS(text, tld='com.ng', lang='en', slow=False)

    # Save the audio file to a temporary file
    speech_file = 'speech.mp3'
    speech.save(speech_file)

    # Play the sound
    playsound('speech.mp3')

    # Remove the file after playing
    os.remove(speech_file)
    
# text_to_speech("Oh my God, what are you doing, stop that!")

In [90]:
def convert_voice_to_text(audio):
    if audio is None:
        return ""
    try:
        text = recognizer.recognize_google(audio)
        print("You said: " + text)
        text_to_speech(f"You said {text}")
    except sr.UnknownValueError:
        text = ""
        print("Sorry I didn't understand that.")
        text_to_speech(f"Sorry I didn't understand that")
    except sr.RequestError as e:
        text = ""
        print("Error: {0}".format(e))
    return text

In [91]:
def process_voice_command(text):
    if "hello" in text.lower():
        print("Hello! How can I help you?")
        text_to_speech(f"Hello! How can I help you")

    elif "what is your name" in text.lower():
        print("My name is Eden")
        text_to_speech(f"My name is Eden")

    elif ("how are you doing today" in text.lower() or
          "how are you" in text.lower() or
          "how are you doing" in text.lower()):
        print("I'm doing alright, thank you very much")
        text_to_speech(f"I'm doing  alright, thank you very much")

    elif ("alright, goodbye" in text.lower() or 
          "alright" in text.lower() or 
          "goodbye" in text.lower() or 
          "all right" in text.lower() or 
          "stop" in text.lower()):
        print("Goodbye! Have a nice day")
        text_to_speech(f"Goodbye! Have a nice day")
        
        return True
    else: 
        print("I didn't understand that command. Please try again.") 
        text_to_speech(f"I didn't understand that command. Please try again.")
    return False

In [92]:
def main():
    end_program = False
    attempts = 0
    max_attempts = 3  # Limit to the number of attempts

    while not end_program and attempts < max_attempts:
        audio = capture_voice_input()
        if audio is None:
            attempts += 1
            print(f"Retrying... ({attempts}/{max_attempts})")
            time.sleep(1)  # Delay to prevent rapid looping
        else:
            text = convert_voice_to_text(audio)
            if text == "":
                attempts += 1
                print(f"Retrying... ({attempts}/{max_attempts})")
            else:
                end_program = process_voice_command(text)

                attempts = 0  # Reset attempts if a valid audio input is processed
    if attempts >= max_attempts:
        print("Too many failed attempts due to timeout. Exiting program.")
        text_to_speech(f"Too many failed attempts due to timeout. Exiting program.")

if __name__ == "__main__":
    main()


Listening...
Sorry I didn't understand that.
Retrying... (1/3)
Listening...
You said: hello
Hello! How can I help you?
Listening...
You said: how are you doing today
I'm doing alright, thank you very much
Listening...
You said: alright goodbye
Goodbye! Have a nice day
